In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import matplotlib.pyplot as plt
# A library for data manipulation and analysis.
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2  # Import l2 regularization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.optimizers import Adam
# Output of plotting commands is displayed inline within the Jupyter notebook.
%matplotlib inline  

# Set a seed so that the results are consistent.
np.random.seed(3) 
# Resets the options
pd.reset_option('all')

2024-06-14 13:18:29.997195: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-14 13:18:29.997327: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-14 13:18:30.148593: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/tmp/ipykernel_18/604412195.py:20: FutureWarning: data_manager option is deprecated and will be removed in a future version. Only the BlockManager will be available.
  pd.reset_option('all')
/tmp/ipykernel_18/604412195.py:20: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating inste

In [3]:
def print_full(x):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', None)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')

In [4]:
def display_full(x):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    display(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')

In [5]:
def display_columns(x):
    pd.set_option('display.max_columns', None)
    display(x)
    pd.reset_option('display.max_columns')

In [6]:
from sklearn.datasets import load_diabetes

diabetes = load_diabetes()
print(diabetes['DESCR'])
adv = pd.DataFrame(diabetes['data'], columns=diabetes['feature_names'])
adv['target'] = diabetes['target']

display_columns(adv)
null_counts = adv.isnull().sum()
print_full(null_counts)

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

Note: Each of these 1

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0
...,...,...,...,...,...,...,...,...,...,...,...
437,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207,178.0
438,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018114,0.044485,104.0
439,0.041708,0.050680,-0.015906,0.017293,-0.037344,-0.013840,-0.024993,-0.011080,-0.046883,0.015491,132.0
440,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044529,-0.025930,220.0


age       0
sex       0
bmi       0
bp        0
s1        0
s2        0
s3        0
s4        0
s5        0
s6        0
target    0
dtype: int64


In [7]:
#print(diabetes)

In [8]:
df_combined = adv.copy()
target_column = 'target'

In [9]:
X = df_combined.drop(columns=[target_column])
y = df_combined[target_column]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
model = Sequential()
model.add(Dense(34, input_dim=X_train.shape[1], activation='sigmoid', kernel_regularizer=l2(0.002)))
model.add(Dense(14, input_dim=X_train.shape[1], activation='sigmoid', kernel_regularizer=l2(0.002)))
model.add(Dense(1, activation='linear', kernel_regularizer=l2(0.005)))  

model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mse'])

history = model.fit(X_train, y_train, epochs=1000, batch_size=10, verbose=2, validation_data=(X_test, y_test))

test_loss, test_mse = model.evaluate(X_test, y_test, verbose=1)

print(f"Test Loss: {test_loss}")
print(f"Test MSE: {test_mse}")

Epoch 1/1000


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


36/36 - 2s - 44ms/step - loss: 29468.4570 - mse: 29468.3711 - val_loss: 26191.9805 - val_mse: 26191.8984
Epoch 2/1000
36/36 - 0s - 3ms/step - loss: 29219.2598 - mse: 29219.1758 - val_loss: 25967.5566 - val_mse: 25967.4688
Epoch 3/1000
36/36 - 0s - 4ms/step - loss: 28998.1777 - mse: 28998.0898 - val_loss: 25772.7422 - val_mse: 25772.6523
Epoch 4/1000
36/36 - 0s - 3ms/step - loss: 28807.8535 - mse: 28807.7559 - val_loss: 25609.6777 - val_mse: 25609.5762
Epoch 5/1000
36/36 - 0s - 3ms/step - loss: 28643.8691 - mse: 28643.7656 - val_loss: 25462.6172 - val_mse: 25462.5117
Epoch 6/1000
36/36 - 0s - 3ms/step - loss: 28497.9512 - mse: 28497.8379 - val_loss: 25329.7051 - val_mse: 25329.5898
Epoch 7/1000
36/36 - 0s - 3ms/step - loss: 28363.3594 - mse: 28363.2441 - val_loss: 25205.7930 - val_mse: 25205.6738
Epoch 8/1000
36/36 - 0s - 3ms/step - loss: 28235.9707 - mse: 28235.8477 - val_loss: 25089.2520 - val_mse: 25089.1230
Epoch 9/1000
36/36 - 0s - 3ms/step - loss: 28112.9707 - mse: 28112.8418 - va

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

model = RandomForestClassifier(n_estimators=95, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

print(f"Test MSE: {mse}")

Test MSE: 4344.404494382023
